In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

print('[+] Feature Selection Methods - Lasso')
#EFFECTIVE FEATURE SELECTION FOR BOTNET DETECTION BASED ON NETWORK FLOW ANALYSIS,2017



[+] Feature Selection Methods - Lasso


In [4]:
# load data
#df = pd.read_csv('./TRAININGS_DATA/lld_lab_iodine_features_added.csv',encoding='utf-8')
#df = pd.read_csv('./TRAININGS_DATA/lld_lab_dnscat_features_added.csv',encoding='utf-8')
df = pd.read_csv('./TRAININGS_DATA/lld_lab_tunnel_features_added.csv',encoding='utf-8')

#df = pd.read_csv('lld_lab_dnscat_4features_added.csv',encoding='utf-8')
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

print("[*] Adding features Feature")
"""
Number of . in subdomain
"""
df['numbdots'] = [x.count('.') for x in df['lld']]

"""
Number of unique character in subdomain
"""
df['numunique'] = [len(set(x)) for x in df['lld']]
x = df.drop(['label','lld'], axis=1)
y = df['label']


[*] Adding features Feature


In [5]:

X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=0)

X_train.shape, X_test.shape

scaler = StandardScaler()
scaler.fit(X_train.fillna(0))

#sel_ = SelectFromModel(LogisticRegression(C=1))
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l2'))
sel_.fit(scaler.transform(X_train.fillna(0)), y_train)

sel_.get_support()

selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 5
selected features: 2
features with coefficients shrank to zero: 0


In [6]:
np.sum(sel_.estimator_.coef_ == 0)

0

In [7]:
removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
removed_feats

Index([], dtype='object')

In [8]:
X_train_selected = sel_.transform(X_train.fillna(0))
X_test_selected = sel_.transform(X_test.fillna(0))
X_train_selected.shape, X_test_selected.shape

((169, 2), (73, 2))

In [9]:
l1_logit = LogisticRegression(C=1, penalty='l2')
l1_logit.fit(scaler.transform(X_train.fillna(0)), y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
np.sum(l1_logit.coef_ == 0)

0